# Data Engineering

In [250]:
import pandas as pd
import sys
import os
sys.path.append("../")
from definitions import ROOT_DIR
import numpy as np

In [251]:
data_folder = os.path.join(ROOT_DIR, 'data/interim')

In [252]:
data = pd.read_csv(os.path.join(data_folder, 'data.csv'))
films = pd.read_csv(os.path.join(data_folder, 'films.csv'))
user = pd.read_csv(os.path.join(data_folder, 'user.csv'))

## Method

My approach is to combine all data together and train a classifier, which will predict rating for a given film and users

## Preparing data

### Processing core dataset

In [253]:
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Timestamp for my approach is redundant

In [254]:
data = data.drop('timestamp', 
                   axis=1, 
                   errors='ignore')

### Processing films

In [255]:
films.head()

,movie_id,movie title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


For now I will not implement a NLP model for processing title and url, so I drop it.

In [256]:
films = films.drop(['movie title', 'IMDb_URL'], 
                   axis=1, 
                   errors='ignore')

I will leave release year only

In [257]:
from sklearn.preprocessing import StandardScaler

films['release_date'] = pd.to_datetime(films['release_date'], format='%d-%b-%Y').dt.year

column_to_normalize = 'release_date'
scaler = StandardScaler()
films[column_to_normalize] = scaler.fit_transform(films[[column_to_normalize]])
films['release_date'] = films['release_date'].fillna(0)

In [258]:
films.head(1)

,movie_id,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0.393978,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Processing users

In [259]:
user.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [260]:
gender_one_hot = pd.get_dummies(user['gender'])
gender_one_hot.columns = ['female', 'male']
user = user.join(gender_one_hot)

In [261]:
occupation_one_hot = pd.get_dummies(user['occupation'])
user = user.join(occupation_one_hot)

In [262]:
user = user.drop(['gender', 'occupation'], 
                   axis=1, 
                   errors='ignore')

In [263]:
user.head()

,user_id,age,zip_code,female,male,administrator,artist,doctor,educator,engineer,...,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer
0,1,24,85711,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2,53,94043,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,23,32067,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,24,43537,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5,33,15213,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


I want to try to extract some useful information from zip code field.

In [264]:
external_data_folder = os.path.join(ROOT_DIR, 'data/external')
zip_data = pd.read_csv(os.path.join(external_data_folder, 'zips1.csv'))

In [265]:
zip_data.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,17126.0,102.6,72001,Adjuntas,"{""72001"": 98.73, ""72141"": 1.27}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37895.0,482.5,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,49136.0,552.4,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5751.0,50.1,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,26153.0,272.1,72011,Añasco,"{""72011"": 96.7, ""72099"": 2.81, ""72083"": 0.37, ...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [266]:
user['zip_code'] = user['zip_code'].apply(lambda x: '00000' if not x.isnumeric() else x)

In [267]:
zip_data_values = set(zip_data['zip'].to_list())
data_zips = set([int(x) for x in user['zip_code'].to_list()])

len(data_zips - zip_data_values)

38

In [268]:
user['zip_code'].nunique()

778

Most of the zip codes are matched. I want to try remove them with coordinates and state name to one-hot-encode it later.

In [269]:
user_zips = [int(x) for x in user['zip_code'].to_list()]
cities = []
coords1, coords2 = [], []

for zip in user_zips:
    row = zip_data.loc[zip_data['zip'] == zip]
    if not len(row):
        cities.append('')
        coords1.append(0)
        coords2.append(0)
        continue
    row = row.iloc[0]
    cities.append(row['state_name'])
    coords1.append(row['lat'])
    coords2.append(row['lng'])

In [270]:
user['state'] = cities
user['lat'] = coords1
user['lng'] = coords2
user = user.drop(['zip_code'], 
                   axis=1, 
                   errors='ignore')

In [271]:
state_one_hot = pd.get_dummies(user['state'])
user = user.join(state_one_hot)

In [272]:
user.head()

,user_id,age,female,male,administrator,artist,doctor,educator,engineer,entertainment,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,1,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,53,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,23,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,33,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now I want to encode age column to different age groups.

In [273]:
bins = [0, 18, 27, 35, 42, 55, float('inf')]
labels = ['0-18', '18-27', '27-35', '35-42', '42-55', '55+']

user['age_range'] = pd.cut(user['age'], bins=bins, labels=labels, right=False)
age_ranges_df = pd.get_dummies(user['age_range'])

# Concatenate the new columns to the original DataFrame
user = pd.concat([user, age_ranges_df], axis=1)

# Drop the original 'age' and 'age_range' columns if needed
user = user.drop(['age', 'age_range', 'state'], axis=1, errors='ignore')

In [274]:
user.head()

,user_id,female,male,administrator,artist,doctor,educator,engineer,entertainment,executive,...,Washington,West Virginia,Wisconsin,Wyoming,0-18,18-27,27-35,35-42,42-55,55+
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Combining all data together

In [275]:
merged_df = pd.merge(data, films, left_on='item_id', right_on='movie_id', how='left')
merged_df = pd.merge(merged_df, user, on='user_id', how='left')
merged_df = merged_df.drop(['movie_id', 'item_id', 'user_id'], axis=1, errors='ignore')

In [276]:
merged_df

,rating,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Washington,West Virginia,Wisconsin,Wyoming,0-18,18-27,27-35,35-42,42-55,55+
0,3,0.534335,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3,0.534335,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0.323799,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2,0.323799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0.534335,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3,0.464156,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
99996,5,-0.307809,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
99997,1,0.253620,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
99998,2,0.464156,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [277]:
result_path = os.path.join(data_folder, 'all.csv')
merged_df.to_csv(result_path, index=False)